In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import kendalltau, spearmanr
from scipy.special import logit, expit

## Задание 1
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
взять в тренировочный набор турниры с dateStart из 2019 года; 
в тестовый — турниры с dateStart из 2020 года.


Data preprocessing

In [2]:
PLAYERS_PATH = "players.pkl"
RESULTS_PATH = "results.pkl"
TOURNAMENTS_PATH = "tournaments.pkl"
RANDOM_STATE = 42

In [3]:
with open(PLAYERS_PATH, 'rb') as f:
    players = pickle.load(f)
with open(RESULTS_PATH, 'rb') as f:
    results = pickle.load(f)
with open(TOURNAMENTS_PATH, 'rb') as f:
    tournaments = pickle.load(f)    

In [4]:
players_df = pd.DataFrame(players.values())
print(players_df)

            id       name  patronymic    surname
0            1    Алексей        None   Абабилов
1           10      Игорь                 Абалов
2           11    Наталья     Юрьевна  Абалымова
3           12      Артур  Евгеньевич    Абальян
4           13       Эрик  Евгеньевич    Абальян
...        ...        ...         ...        ...
204058  224700      Артём  Евгеньевич      Садов
204059  224701     Даниил    Олегович   Трефилов
204060  224702   Владимир  Араратович   Басенцян
204061  224703     Руслан   Ринатович   Дауранов
204062  224704  Александр  Викторович    Гапонов

[204063 rows x 4 columns]


In [5]:
tournaments_df = pd.DataFrame(tournaments.values())
tournaments_df['year'] = list(tournaments_df['dateStart'].astype(str).str[:4])
tournaments_df.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty,year
0,1,Чемпионат Южного Кавказа,2003-07-25T00:00:00+04:00,2003-07-27T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None,2003
1,2,Летние зори,2003-08-09T00:00:00+04:00,2003-08-09T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None,2003
2,3,Турнир в Ижевске,2003-11-22T00:00:00+03:00,2003-11-24T00:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None,2003
3,4,Чемпионат Украины. Переходной этап,2003-10-11T00:00:00+04:00,2003-10-12T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None,2003
4,5,Бостонское чаепитие,2003-10-10T00:00:00+04:00,2003-10-13T00:00:00+04:00,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None,2003


In [6]:
results_list = list(results.values())
df_list = []
for i in range(len(results_list)):
    df = pd.DataFrame(results_list[i])
    df['id'] = tournaments_df.iloc[i]["id"]
    if len(df) > 0:
        df_list.append(df)
results_df = pd.concat(df_list, ignore_index= True, axis=0)
results_df.head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,id
0,"{'id': 242, 'name': 'Команда Азимова', 'town':...",None,"{'name': 'Команда Азимова', 'town': {'id': 21,...",0.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",1
1,"{'id': 640, 'name': 'Перезагрузка', 'town': {'...",None,"{'name': 'Перезагрузка', 'town': {'id': 100, '...",0.0,None,2.0,[],[],[],1
2,"{'id': 245, 'name': 'Айастан', 'town': {'id': ...",None,"{'name': 'Айастан', 'town': {'id': 100, 'name'...",0.0,None,3.5,[],[],[],1
3,"{'id': 299, 'name': 'Команда Гусейнова', 'town...",None,"{'name': 'Команда Гусейнова', 'town': {'id': 2...",0.0,None,3.5,[],[],[],1
4,"{'id': 1189, 'name': 'Грааль', 'town': {'id': ...",None,"{'name': 'Грааль', 'town': {'id': 369, 'name':...",0.0,None,5.0,[],[],[],1


In [7]:
tournaments_df = tournaments_df[(tournaments_df['year'] == "2019") | (tournaments_df['year'] == "2020")]
tournaments_ids = list(tournaments_df['id'].unique())
ids_train = list(tournaments_df.loc[(tournaments_df['year'] == "2019"),"id"].unique())
ids_test = list(tournaments_df.loc[(tournaments_df['year'] == "2020"),"id"].unique())
results_df = results_df[results_df['mask'].isna() == False]
#results_df = results_df[results_df['mask'].str.contains('^[0-1]$')]
results_df = results_df[results_df['teamMembers'].str.len() > 0]
results_df = results_df[results_df['team'].str.len() > 0]
results_df = results_df.query("id in @tournaments_ids")
results_train = results_df.query("id in @ids_train")
results_test = results_df.query("id in @ids_test")
tournaments_ids = list(results_df['id'].unique())
print(f"Size of tournaments = {len(tournaments_ids)}")

Size of tournaments = 848


In [198]:
def make_dataset(results_df, testing = False):
    df_list = []
    for i, row in results_df.iterrows():
        team_members_ids = [(x['player']['id'], x['player']['name'] + " " + x['player']['surname']) for x in row['teamMembers']]
        l = [(t == '0') | (t == '1') for t in list(row['mask'])]
        if sum(l) != len(l):
            continue
        number_of_questions = len(list(row['mask']))
        for k in team_members_ids:
            if testing  == False:
                for j in range(number_of_questions):
                    col_name = (str(row['id']) + '_' + str(j))
                    df_list.append((row["id"], k[0], row["team"]["id"], k[1], col_name, int(row['mask'][j])))
            else:
                df_list.append((row["id"], k[0], row["team"]["id"], k[1], str(row['id']), int(row['questionsTotal'])))
    df = pd.DataFrame(df_list, columns=["tournament_id", "player_id","team_id", "player_name", "q_no", "score"])
    return df

In [199]:
df_train = make_dataset(results_train)
df_test = make_dataset(results_test, True)

In [10]:
ids_train = df_train['player_id'].unique()
ids_test = df_test['player_id'].unique()
print(f"train players = {len(ids_train)}")
print(f"test players = {len(ids_test)}")

train players = 57288
test players = 28127


In [11]:
df_train

,player_id,team_id,player_name,q_no,score
0,6212,45556,Юрий Выменец,4772_0,1
1,6212,45556,Юрий Выменец,4772_1,1
2,6212,45556,Юрий Выменец,4772_2,1
3,6212,45556,Юрий Выменец,4772_3,1
4,6212,45556,Юрий Выменец,4772_4,1
...,...,...,...,...,...
17295085,217156,76130,Юлия Рябкова,6255_31,0
17295086,217156,76130,Юлия Рябкова,6255_32,0
17295087,217156,76130,Юлия Рябкова,6255_33,0
17295088,217156,76130,Юлия Рябкова,6255_34,0


In [12]:
df_test

,player_id,team_id,player_name,q_no,score
0,18490,66120,Дмитрий Литвинов,5414,33
1,116901,66120,Екатерина Шевцова,5414,33
2,8532,66120,Евгений Гурт,5414,33
3,42346,66120,Богдана Романцова,5414,33
4,123190,66120,Елена Красникова,5414,33
...,...,...,...,...,...
99651,129706,69918,Алексей Трилис,6456,16
99652,192901,69918,Мария Федоркина,6456,16
99653,165962,63129,Павел Горбиль,6456,13
99654,154624,63129,Артём Пильненький,6456,13


## Задание 2
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
 - повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
 - в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
 - для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


Будем использовать логистическую регрессию для предсказания вероятности ответа игрока на тот или иной вопрос. Рейтинг игрока и сложность вопроса будем определять по весам обученной регресии. 

In [13]:
encoder = OneHotEncoder(handle_unknown='ignore')

X_train = encoder.fit_transform(df_train[['player_id', 'q_no']])
X_test = encoder.transform(df_test[['player_id', 'q_no']])
y_train = df_train["score"]

In [14]:
df_train

,player_id,team_id,player_name,q_no,score
0,6212,45556,Юрий Выменец,4772_0,1
1,6212,45556,Юрий Выменец,4772_1,1
2,6212,45556,Юрий Выменец,4772_2,1
3,6212,45556,Юрий Выменец,4772_3,1
4,6212,45556,Юрий Выменец,4772_4,1
...,...,...,...,...,...
17295085,217156,76130,Юлия Рябкова,6255_31,0
17295086,217156,76130,Юлия Рябкова,6255_32,0
17295087,217156,76130,Юлия Рябкова,6255_33,0
17295088,217156,76130,Юлия Рябкова,6255_34,0


In [15]:
log_reg = LogisticRegression(solver='liblinear', class_weight = 'balanced', random_state=RANDOM_STATE)
log_reg.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=42, solver='liblinear')

In [162]:
rating_train_df = pd.DataFrame()
rating_train_df['player_id'] = np.unique(df_train['player_id'])
names = []
for id in np.unique(df_train['player_id']):
    res = players_df[players_df['id'] == id]
    names.append(list(res['name'])[0] + " " + list(res['surname'])[0])
rating_train_df['player_name'] = names
rating_train_df['rating'] = log_reg.coef_[0][:len(rating_train_df)]

In [17]:
rating_train_df = rating_train_df.sort_values(by='rating', ascending=False).reset_index(drop=True)
rating_train_df.head(11)

,player_id,player_name,rating
0,27403,Максим Руссо,4.034077
1,4270,Александра Брутер,3.901689
2,28751,Иван Семушин,3.872321
3,27822,Михаил Савченков,3.730015
4,30270,Сергей Спешков,3.726224
5,30152,Артём Сорожкин,3.724415
6,18036,Михаил Левандовский,3.614449
7,87637,Антон Саксонов,3.583751
8,20691,Станислав Мереминский,3.552085
9,22799,Сергей Николенко,3.542753


## Задание 3
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
 - предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
 - в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


вероятность команды ответить верно на вопрос -- это вероятность того, что хотя бы один из игроков ответит верно или 1 - (вероятность того, что все ошибутся), предполагая что игроки отвечают независимо, получаем 1 - произведение вероятностей ошибиться. Вероятность игрока ответить на вопрос верно будет считать как сигмоиду от суммы рейтинга игрока и веса смещения

In [18]:
# Предсказываем вероятности
predictions = log_reg.predict_proba(X_test)[:,1]

In [19]:
prediction_df = df_test.copy()
prediction_df["answer_p"] = 1 - predictions 
prediction_df["answer_p"] = prediction_df.groupby(['q_no', 'team_id'])['answer_p'].transform(lambda x: 1 - np.prod(x))
prediction_df

,player_id,team_id,player_name,q_no,score,answer_p
0,18490,66120,Дмитрий Литвинов,5414,33,0.999944
1,116901,66120,Екатерина Шевцова,5414,33,0.999944
2,8532,66120,Евгений Гурт,5414,33,0.999944
3,42346,66120,Богдана Романцова,5414,33,0.999944
4,123190,66120,Елена Красникова,5414,33,0.999944
...,...,...,...,...,...,...
99651,129706,69918,Алексей Трилис,6456,16,0.973230
99652,192901,69918,Мария Федоркина,6456,16,0.973230
99653,165962,63129,Павел Горбиль,6456,13,0.508641
99654,154624,63129,Артём Пильненький,6456,13,0.508641


In [20]:
prediction_df.sort_values(by=['q_no', 'score'], ascending=False, inplace=True)
prediction_df["real_rank"] = prediction_df.groupby('q_no')['score'].transform(lambda x: np.arange(1, len(x) + 1))
prediction_df.sort_values(by=['q_no', 'answer_p'], ascending=False, inplace=True)
prediction_df["pred_rank"] = prediction_df.groupby('q_no')['answer_p'].transform(lambda x: np.arange(1, len(x) + 1))
spearman_coeff = prediction_df.groupby('q_no').apply(lambda x: spearmanr(x['real_rank'], x['pred_rank']).correlation).mean()
kendall_coeff = prediction_df.groupby('q_no').apply(lambda x: kendalltau(x['real_rank'], x['pred_rank']).correlation).mean()
print(f'Spearman coefficient is = {spearman_coeff}')
print(f'Kendall coefficient is = {kendall_coeff}')

Spearman coefficient is = 0.7863966951486834
Kendall coefficient is = 0.6295793605490344


Значения корреляции у бейзлайна в пределах нормы

## Задание 4
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


Скрытой переменной будет маска ответов игрока, тогда ответ команды является повопросным или ответов всех игроков, который мы и наблюдаем в данных. 

In [183]:
IT_NUM = 2
EPS = 1e-3

In [184]:
class baseline:
    def __init__(self, model ,baseline_model):
        self.alg = model
        self.alg.coef_ = baseline_model.coef_
        self.alg.intercept_ = baseline_model.intercept_

    def fit(self, X_train, y_train):
        self.alg.fit(X_train, logit(y_train))

    def predict_proba(self, X_train):
        res = self.alg.predict(X_train)
        res = expit(res)
        return res


In [185]:
def E_step(preds, df):
    df['preds'] = 1 - preds
    prob = df.groupby(['q_no','team_id']).agg({'preds': 'prod'}).reset_index()
    prob['preds'] = 1 - prob['preds']
    prob = df[[ 'q_no','team_id']].merge(prob)
    res = preds / prob['preds']
    res = res  * df['score']
    res = np.clip(res, EPS, 1 - EPS).values
    return res

In [186]:
def M_step(X_train, y_train, baseline_alg):
   baseline_alg.fit(X_train, y_train)
   return baseline_alg

In [187]:
def calc_metric(df_test, predictions):
    prediction_df = df_test.copy()
    prediction_df["answer_p"] = 1 - predictions 
    prediction_df["answer_p"] = prediction_df.groupby(['q_no', 'team_id'])['answer_p'].transform(lambda x: 1 - np.prod(x))
    prediction_df.sort_values(by=['q_no', 'score'], ascending=False, inplace=True)
    prediction_df["real_rank"] = prediction_df.groupby('q_no')['score'].transform(lambda x: np.arange(1, len(x) + 1))
    prediction_df.sort_values(by=['q_no', 'answer_p'], ascending=False, inplace=True)
    prediction_df["pred_rank"] = prediction_df.groupby('q_no')['answer_p'].transform(lambda x: np.arange(1, len(x) + 1))
    spearman_coeff = prediction_df.groupby('q_no').apply(lambda x: spearmanr(x['real_rank'], x['pred_rank']).correlation).mean()
    kendall_coeff = prediction_df.groupby('q_no').apply(lambda x: kendalltau(x['real_rank'], x['pred_rank']).correlation).mean()
    print(f'Spearman coefficient is = {spearman_coeff}')
    print(f'Kendall coefficient is = {kendall_coeff}')
    return spearman_coeff, kendall_coeff

In [193]:
def EM_alg(df_train, df_train_enc, df_test, df_test_enc, baseline_alg):
    for iter in range(IT_NUM):
        print(f"processing {iter+1} iteration")
        predictions = baseline_alg.predict_proba(df_train_enc).flatten()
        new_preds = E_step(predictions, df_train)
        baseline_alg = M_step(df_train_enc, new_preds, baseline_alg)
        predictions = baseline_alg.predict_proba(df_test_enc)
        calc_metric(df_test, predictions)

In [194]:
baseline_alg = baseline(LinearRegression(), log_reg)

In [195]:
EM_alg(df_train, X_train, df_test, X_test, baseline_alg)

processing 1 iteration
Spearman coefficient is = 0.7940020388566855
Kendall coefficient is = 0.6379695127594672
processing 2 iteration
Spearman coefficient is = 0.7974250796075547
Kendall coefficient is = 0.6426022132465625


##  Задание 5
А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [210]:
tournaments_df = tournaments_df.set_index("id")

In [240]:
questions = set(np.unique(df_train['q_no']))
predictions = baseline_alg.alg.coef_[len(rating_train_df):]
#tournaments_df = tournaments_df.set_index("id")
rating = dict(zip(questions, predictions))
df_train['complexity'] = df_train['q_no'].map(rating)
df_train['tournament_name'] = list(tournaments_df.loc[df_train['tournament_id'], "name"])

In [241]:
tournaments_rating = df_train[['tournament_name', 'q_no', 'complexity']].drop_duplicates()
tournaments_rating = pd.DataFrame(tournaments_rating.groupby('tournament_name')['complexity'].mean())
tournaments_with_complexity = tournaments_rating.sort_values(by='complexity', ascending=False).reset_index()

In [242]:
tournaments_with_complexity.head(5)

,tournament_name,complexity
0,ТРИОтлон-3,2.250399
1,Синхрон Кеста,2.002426
2,Знатокиада. Синхрон Олимпийского турнира,1.932235
3,Из Минска с любовью. Этап 5.,1.742357
4,Кубок Хайфы,1.674188


In [243]:
tournaments_with_complexity.tail(5)

,tournament_name,complexity
605,Californication,-1.408363
606,Скрытые фигуры,-1.467246
607,Загадочный ларец,-1.527956
608,Эврика! - 11,-1.656879
609,Играй для Жизни,-1.755646


Кажется получилось не так классно как хотелось....